# Combining & Merging datasets

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

Data contained in pandas objects can be combined together in a number of ways:

1. [**Database-Style DataFrame Joins**](#Database-Style-DataFrame-Joins)

    - [**Merge**](#Merge) [`pd.merge(df1, df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge): for combining data on common columns or indices

    - [**Join**](#Join) [`df1.join(df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join): for combining data on a key column or an index
    

2. [**Concatenating Along an Axis**](#Concatenating-Along-an-Axis)

    - [**Concat**](#Concat) [`pd.concat([df1, df2])`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat): for combining DataFrames across rows or columns

    - [**Append**](#Append) [`df1.append(df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html): stacking vertically

------------------------------------------------------------------------------------------------------------------------------

## Database-Style DataFrame Joins

# Merge

Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on. When performing a cross merge, no column specifications to merge on are allowed.

When you use merge(), you’ll provide two required arguments:

- The left DataFrame
- The right DataFrame

After that, you can provide a number of optional arguments to define how your datasets are merged:

**how**: This defines what kind of merge to make. It defaults to 'inner', but other possible options include 'outer', 'left', and 'right'.

**on**: Use this to tell merge() which columns or indices (also called key columns or key indices) you want to join on. This is optional. If it isn’t specified, and left_index and right_index (covered below) are False, then columns from the two DataFrames that share names will be used as join keys. If you use on, then the column or index you specify must be present in both objects.

**left_on and right_on**: Use either of these to specify a column or index that is present only in the left or right objects that you are merging. Both default to None.

**left_index and right_index**: Set these to True to use the index of the left or right objects to be merged. Both default to False.

**suffixes**: This is a tuple of strings to append to identical column names that are not merge keys. This allows you to keep track of the origins of columns with the same name.

### Joining (Merging) DataFrames

Using the [MovieLens 100k data](http://grouplens.org/datasets/movielens/), let's create two DataFrames:

- **movies**: shows information about movies, namely a unique **movie_id** and its **title**
- **ratings**: shows the **rating** that a particular **user_id** gave to a particular **movie_id** at a particular **timestamp**

In [ ]:
#first things first

#### Movies

In [ ]:
url = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.item"

In [ ]:
movie_cols = ['movie_id', 'title']
movies = pd.read_csv(url, sep='|', header=None, names=movie_cols, usecols=[0, 1])

#### Ratings

In [ ]:
url1 = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.data"

In [ ]:
rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(url1, sep='\t', header=None, names=rating_cols)

#### Merging Movies and Ratings

Let's pretend that you want to examine the ratings DataFrame, but you want to know the **title** of each movie rather than its **movie_id**. The best way to accomplish this objective is by "joining" (or "merging") the DataFrames using the Pandas `merge` function:

Here's what just happened:

- Pandas noticed that movies and ratings had one column in common, namely **movie_id**. This is the "key" on which the DataFrames will be joined.
- The first **movie_id** in movies is 1. Thus, Pandas looked through every row in the ratings DataFrame, searching for a movie_id of 1. Every time it found such a row, it recorded the **user_id**, **rating**, and **timestamp** listed in that row. In this case, it found 452 matching rows.
- The second **movie_id** in movies is 2. Again, Pandas did a search of ratings and found 131 matching rows.
- This process was repeated for all of the remaining rows in movies.

At the end of the process, the movie_ratings DataFrame is created, which contains the two columns from movies (**movie_id** and **title**) and the three other colums from ratings (**user_id**, **rating**, and **timestamp**).

- **movie_id** 1 and its **title** are listed 452 times, next to the **user_id**, **rating**, and **timestamp** for each of the 452 matching ratings.
- **movie_id** 2 and its **title** are listed 131 times, next to the **user_id**, **rating**, and **timestamp** for each of the 131 matching ratings.
- And so on, for every movie in the dataset.

Notice the shapes of the three DataFrames:

- There are 1682 rows in the movies DataFrame.
- There are 100000 rows in the ratings DataFrame.
- The `merge` function resulted in a movie_ratings DataFrame with 100000 rows, because every row from ratings matched a row from movies.
- The movie_ratings DataFrame has 5 columns, namely the 2 columns from movies, plus the 4 columns from ratings, minus the 1 column in common.

By default, the `merge` function joins the DataFrames using all column names that are in common (**movie_id**, in this case). The [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge.html) explains how you can override this behavior.

**What if the columns you want to join on don't have the same name?**

#### What if you want to join on one index?

#### What if you want to join on two indexes?

### Types of join

There are actually four types of joins supported by the Pandas `merge` function. Here's how they are described by the documentation:

- **inner:** use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys
- **outer:** use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically
- **left:** use only keys from left frame, similar to a SQL left outer join; preserve key order
- **right:** use only keys from right frame, similar to a SQL right outer join; preserve key order

The default is the "inner join", which was used when creating the movie_ratings DataFrame.

It's easiest to understand the different types by looking at some simple examples:

![join](https://www.dofactory.com/img/sql/sql-joins.png)

In [ ]:
A = pd.DataFrame({'color': ['green', 'yellow', 'red'], 'num':[1, 2, 3]})
A

In [ ]:
B = pd.DataFrame({'color': ['green', 'yellow', 'pink'], 'size':['S', 'M', 'L']})
B

#### Inner join

Only include observations found in both A and B:

#### Outer join

Include observations found in either A or B:

#### Left join

Include all observations found in A:

#### Right join

Include all observations found in B:

# Join

Join columns of another DataFrame.

Join columns with other DataFrame either on index or on a key column. Efficiently join multiple DataFrame objects by index at once by passing a list.

If we want to join using the key columns, we need to set key to be the index in both df and other. The joined DataFrame will have key as its index.

While `merge` is a module function, `.join` is an object function that lives on your DataFrame. This enables you to specify only one DataFrame, which will join the DataFrame you call `.join` on.

Under the hood, `.join` uses `merge`

In [ ]:
#con las películas


In [ ]:
#si tienen mismo index, igual que al hacer pd.merge(movies_index, ratings_index, left_index=True, right_index=True)


# Concatenating Along an Axis

# Concat

Concatenate pandas objects along a particular axis with optional set logic along the other axes.

Can also add a layer of hierarchical indexing on the concatenation axis, which may be useful if the labels are the same (or overlapping) on the passed axis number.

Concatenation is a bit different from the merging techniques you saw above. With merging, you can expect the resulting dataset to have rows from the parent datasets mixed in together, often based on some commonality. Depending on the type of merge, you might also lose rows that don’t have matches in the other dataset.

With concatenation, your datasets are just stitched together along an axis — either the row axis or column axis. Visually, a concatenation with no parameters along rows would look like this:

![](https://files.realpython.com/media/concat_axis0.2ec65b5f72bc.png)

To implement this in code, you’ll use concat() and pass it a list of DataFrames that you want to concatenate. 

**objs**: This parameter takes any sequence (typically a list) of Series or DataFrame objects to be concatenated. You can also provide a dictionary. In this case, the keys will be used for the keys options.

**axis**: Like in the other techniques, this represents the axis you will concatenate along. The default value is 0, which concatenates along the index (or row axis), while 1 concatenates along columns (vertically). You can also use the string values index or columns.

**join**: This is similar to the how parameter in the other techniques, but it only accepts the values inner or outer. The default value is outer, which preserves data, while inner would eliminate data that does not have a match in the other dataset.

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index = ["a", "b", "c"], columns = ["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index = ["a", "c"], columns = ["three", "four"])

A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the `keys` argument.

If you pass a dict of objects instead of a list, the dict's keys will be used for the keys option: 


In [ ]:
#usando movies y ratings vemos que para este problema concat no sirve


# Append

Because direct array concatenation is so common, ``Series`` and ``DataFrame`` objects have an ``append`` method that can accomplish the same thing in fewer keystrokes.

Append is the specific case `(axis=0, join='outer')` of concat

For example, rather than calling ``pd.concat([df1, df2])``, you can simply call ``df1.append(df2)``:

In [ ]:
df_a = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df2_a = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))

In [ ]:
# concat también tiene el parámetro ignore_index = True
